In [0]:
import json
import random
import datetime
import os
import time

class GeradorDadosFakeVendas:
    categorias = ["Camisas", "Calças", "Sapatos", "Colares", "Jaquetas"]
    marcas = ["Marca A", "Marca B", "Marca C", "Marca D", "Marca E", "Marca F", "Marca G"]
    cores = ["Vermelho", "Azul", "Verde", "Preto", "Branco", "Amarelo", "Rosa","Marrom","Roxo"]
    categoria_singular = {
        "Camisas": "Camisa",
        "Calças": "Calça",
        "Sapatos": "Sapato",
        "Colares": "Colar",
        "Jaquetas": "Jaqueta"
    }

    def __init__(self, volume_base_path, num_produtos=100, num_vendas_iniciais=5000):
        self.volume_base_path = volume_base_path
        self.sales_dir = os.path.join(volume_base_path, "vendas")
        self.products_dir = os.path.join(volume_base_path, "produtos")
        self.checkpoint_path = os.path.join(volume_base_path, "checkpoint.json")
        self.num_produtos = num_produtos
        self.num_vendas_iniciais = num_vendas_iniciais
        self.produtos = []
        self.vendas_dict = {}
        self.ultimo_id_venda = 0
        self.ultimo_id_prod = 0
        self.carregar_checkpoint()

    def carregar_checkpoint(self):
        if os.path.exists(self.checkpoint_path):
            with open(self.checkpoint_path, 'r', encoding='utf-8') as f:
                checkpoint = json.load(f)
                self.produtos = checkpoint["produtos"]
                self.vendas_dict = checkpoint["vendas_dict"]
                self.ultimo_id_venda = checkpoint["ultimo_id_venda"]
                self.ultimo_id_prod = checkpoint["ultimo_id_prod"]

    def salvar_checkpoint(self):
        checkpoint = {
            "produtos": self.produtos,
            "vendas_dict": self.vendas_dict,
            "ultimo_id_venda": self.ultimo_id_venda,
            "ultimo_id_prod": self.ultimo_id_prod
        }
        with open(self.checkpoint_path, 'w', encoding='utf-8') as f:
            json.dump(checkpoint, f)
    def gerar_catalogo_produtos(self):
        produtos = []
        nomes_usados = set()
        for pid in range(1, self.num_produtos + 1):
            categoria = random.choice(self.categorias)
            marca = random.choice(self.marcas)
            cor = random.choice(self.cores)
            tipo = self.categoria_singular.get(categoria, categoria.rstrip('s'))
            nome = f"{tipo} {cor} {marca}"
            while nome in nomes_usados:
                categoria = random.choice(self.categorias)
                marca = random.choice(self.marcas)
                cor = random.choice(self.cores)
                tipo = self.categoria_singular.get(categoria, categoria.rstrip('s'))
                nome = f"{tipo} {cor} {marca}"
            nomes_usados.add(nome)
            preco = round(random.uniform(20, 500), 2)
            produto = {"id_produto": pid, "nome_produto": nome, "categoria": categoria, "marca": marca, "preco_lista": preco}
            produtos.append(produto)
        return produtos

    def gerar_vendas(self, inicio, fim, prox_id, num_vendas):
        vendas = []
        tempo_total = (fim - inicio).total_seconds()
        for _ in range(num_vendas):
            offset = random.randrange(int(tempo_total)) if tempo_total > 0 else 0
            momento = inicio + datetime.timedelta(seconds=offset)
            timestamp_str = momento.strftime("%Y-%m-%d %H:%M:%S")
            num_itens = random.choices([1, 2, 3, 4, 5, 6], weights=[50, 25, 15, 5, 4, 1])[0]
            itens_venda = []
            for _ in range(num_itens):
                produto = random.choice(self.produtos)
                preco_base = produto["preco_lista"]
                preco_venda = round(preco_base * random.uniform(0.9, 1.1), 2)
                preco_str = f"R$ {preco_venda:.2f}".replace('.', ',')
                item = {"id_produto": produto["id_produto"], "quantidade": random.randint(1, 3), "preco_unitario": preco_str}
                itens_venda.append(item)
                if num_itens == 6: ## Fazendo o resultado vir propositalmente vazio, forçando caso de venda inválida
                    itens_venda = []
            venda = {"id_transacao": prox_id, "timestamp": timestamp_str, "forma_pagamento": 
                random.choices(["Cartão de Crédito", "Cartão de Débito", "Dinheiro", "Pix", "Boleto", None], weights=[40, 22, 7, 28, 1, 2])[0], 
                "cancelado": 0, "itens": itens_venda}
            vendas.append(venda)
            prox_id += 1
        return vendas, prox_id - 1

    def carga_inicial(self):
        os.makedirs(self.sales_dir, exist_ok=True)
        os.makedirs(self.products_dir, exist_ok=True)
        self.produtos = self.gerar_catalogo_produtos()
        fim = datetime.datetime.now() - datetime.timedelta(days=1)
        inicio = fim - datetime.timedelta(days=30)
        vendas, self.ultimo_id_venda = self.gerar_vendas(inicio, fim, 1, self.num_vendas_iniciais)
        self.ultimo_id_prod = self.num_produtos
        with open(os.path.join(self.sales_dir, "vendas_inicial.json"), 'w', encoding='utf-8') as f_v:
            for reg in vendas:
                f_v.write(json.dumps(reg) + "\n")
        with open(os.path.join(self.products_dir, "produtos_inicial.json"), 'w', encoding='utf-8') as f_p:
            for prod in self.produtos:
                f_p.write(json.dumps(prod) + "\n")
        self.vendas_dict = {reg["id_transacao"]: reg for reg in vendas}

    def gerar_incremental(self, novas_vendas=25, atualizacoes_cancel=5, novos_produtos=None, atualizacoes_prod=None):
        agora = datetime.datetime.now()
        uma_hora_atras = agora - datetime.timedelta(hours=1)

        if novos_produtos is None:
            novos_produtos = random.randint(0, 3)
        if atualizacoes_prod is None:
            atualizacoes_prod = random.randint(1, 5)

        novas_vendas = random.randint(int(novas_vendas*0.5), int(1.5*novas_vendas))
        vendas_novas, novo_ultimo_id = self.gerar_vendas(uma_hora_atras, agora, self.ultimo_id_venda + 1, novas_vendas)
        self.ultimo_id_venda = novo_ultimo_id

        vendas_ativas = [vid for vid, reg in self.vendas_dict.items() if reg["cancelado"] == 0]
        random.shuffle(vendas_ativas)
        ids_cancelar = vendas_ativas[:atualizacoes_cancel]

        vendas_canceladas = []
        for vid in ids_cancelar:
            registro_cancelado = self.vendas_dict[vid].copy()
            registro_cancelado["cancelado"] = 1
            vendas_canceladas.append(registro_cancelado)
            self.vendas_dict[vid]["cancelado"] = 1  # Atualiza também em memória

        novos_produtos_lista = []
        for _ in range(novos_produtos):
            categoria = random.choice(self.categorias)
            marca = random.choice(self.marcas)
            cor = random.choice(self.cores)
            tipo = self.categoria_singular.get(categoria, categoria.rstrip('s'))
            nome = f"{tipo} {cor} {marca}"
            preco = round(random.uniform(20, 500), 2)
            produto = {
                "id_produto": self.ultimo_id_prod + 1,
                "nome_produto": nome,
                "categoria": categoria,
                "marca": marca,
                "preco_lista": preco
            }
            novos_produtos_lista.append(produto)
            self.produtos.append(produto)
            self.ultimo_id_prod += 1

        ids_existentes = list(range(1, self.ultimo_id_prod - novos_produtos + 1))
        random.shuffle(ids_existentes)
        ids_atualizar = ids_existentes[:atualizacoes_prod]
        produtos_atualizados_lista = []
        for pid in ids_atualizar:
            produto = self.produtos[pid - 1]
            produto["preco_lista"] = round(produto["preco_lista"] * random.uniform(0.9, 1.1), 2)
            produtos_atualizados_lista.append(produto)

        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

        with open(os.path.join(self.sales_dir, f"vendas_incremental_{timestamp}.json"), 'w', encoding='utf-8') as f_v:
            for reg in vendas_novas + vendas_canceladas:
                f_v.write(json.dumps(reg) + "\n")

        with open(os.path.join(self.products_dir, f"produtos_incremental_{timestamp}.json"), 'w', encoding='utf-8') as f_p:
            for prod in novos_produtos_lista + produtos_atualizados_lista:
                f_p.write(json.dumps(prod) + "\n")

    def verificar_e_gerar_dados(self):
        if not os.path.exists(self.sales_dir) or not os.listdir(self.sales_dir):
            print("Diretório vazio ou inexistente. Gerando carga inicial...")
            self.carga_inicial()
        else:
            print("Dados existentes. Gerando carga incremental...")
            self.gerar_incremental()
        self.salvar_checkpoint()

    def stream(self, num_lotes, intervalo=15):
        for lote in range(num_lotes):
            print(f"Gerando lote {lote+1}/{num_lotes}...")
            self.gerar_incremental()
            self.salvar_checkpoint()
            time.sleep(intervalo)